In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# train_data = pd.read_csv("E:/COLUMBIA MS/FALL 2023/AML - Vijay Pappu/AML PROJECT/train_data.csv", chunksize = 100000)
# train_labels = pd.read_csv("E:/COLUMBIA MS/FALL 2023/AML - Vijay Pappu/AML PROJECT/train_labels.csv")

### Converting datatypes of columns to int32, int16, and int8 (where possible)

In [3]:
train_data = pd.read_csv("E:/COLUMBIA MS/FALL 2023/AML - Vijay Pappu/AML PROJECT/train_data.csv", chunksize = 100000)
train_labels = pd.read_csv("E:/COLUMBIA MS/FALL 2023/AML - Vijay Pappu/AML PROJECT/train_labels.csv")
df2 = pd.DataFrame()
for df in train_data:
    cols = ['customer_ID', 'S_2', 'B_30', 'B_38', 'D_63', 'D_64', 'D_66', 'D_68', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126']
    df['customer_ID'] = df['customer_ID'].apply(lambda x: int(x[-16:],16) ).astype('int32')
    df['S_2'] = pd.to_datetime(df['S_2'])
    df['B_30'] = df['B_30'].fillna(-1).astype('int8')
    df['B_38'] = df['B_38'].fillna(-1).astype('int8')
    df['D_63'] = df['D_63'].astype('category').cat.codes.astype('int8')
    df['D_64'] = df['D_64'].astype('category').cat.codes.astype('int8')
    df['D_66'] = df['D_66'].astype('category').cat.codes.astype('int8')
    df['D_68'] = df['D_68'].astype('category').cat.codes.astype('int8')
    df['D_114'] = df['D_114'].fillna(-1).astype('int8')
    df['D_116'] = df['D_116'].fillna(-1).astype('int8')
    df['D_117'] = df['D_117'].fillna(-1).astype('int8')
    df['D_120'] = df['D_120'].fillna(-1).astype('int8')
    df['D_126'] = df['D_126'].fillna(-1).astype('int8')
    for c in df.columns:
        if c not in cols:
            df[c] = df[c].fillna(-1).astype('int16')
#     print("here")
    df2 = pd.concat([df2, df])

In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: datetime64[ns](1), int16(177), int32(1), int8(11)
memory usage: 1.9 GB


In [5]:
df2.sample(5)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
1278831,-1607926392,2018-02-10,0,0,0,0,0,0,0,0,...,-1,-1,-1,0,0,0,-1,0,0,0
2585212,-126383744,2018-03-23,0,0,0,0,0,-1,0,0,...,-1,-1,-1,0,0,0,-1,0,0,0
5400717,-1019104863,2017-07-16,0,0,0,1,0,0,0,0,...,-1,-1,-1,1,0,0,0,1,0,0
1963659,1645201032,2017-12-13,0,0,0,1,0,0,0,0,...,-1,-1,-1,0,0,0,-1,0,0,0
3495935,865710414,2017-11-02,0,0,0,0,0,0,0,0,...,-1,-1,-1,0,0,0,-1,0,0,0


In [6]:
train_labels.sample(5)

,customer_ID,target
87151,30812ddc387c7e16b2a049ea4c62d2e1e0b879675a8640...,0
218775,79c4940bc1e7ff05b8a18672e457b5e407459d72f6debc...,0
274857,990e2413e493cfe362cb4c81ba88fb5daf23e211f3a163...,0
320347,b267be8098a1279b53f021fb3b256e05935a994e520226...,0
242688,8719044e57273158b02257350650441a42877202ecf2fc...,1


In [7]:
print("Total number of rows in train_data : ", len(df2))
print("Total number of unique customers in train_data : ", len(df2['customer_ID'].unique()))

Total number of rows in train_data :  5531451
Total number of unique customers in train_data :  458884


In [15]:
len(train_labels)

458913

### Extracting Numeric Columns to apply min, max, mean, std aggregate functions

In [8]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
time_col = ['S_2']
id_col = ['customer_ID']
numeric_cols = []
for col in df2.columns:
    if col not in cat_cols and col not in time_col and col not in id_col:
        numeric_cols.append(col)

print(len(numeric_cols))

177


In [9]:
columns_to_aggregate = numeric_cols 

# Define aggregation functions for each column
aggregation_functions = {
    col: ['mean', 'std', 'min', 'max'] for col in columns_to_aggregate
}

# Group by 'customer_id' and apply the aggregation functions
aggregated_df = df2.groupby('customer_ID').agg(aggregation_functions).reset_index()

# Rename columns for clarity
aggregated_df.columns = [f'{col}_{agg}' for col, agg in aggregated_df.columns]

In [10]:
aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458884 entries, 0 to 458883
Columns: 709 entries, customer_ID_ to D_145_max
dtypes: float64(354), int16(354), int32(1)
memory usage: 1.5 GB


In [11]:
aggregated_df.columns
aggregated_df.rename(columns={'customer_ID_': 'customer_ID'}, inplace=True)
train_labels['customer_ID'] =train_labels['customer_ID'].apply(lambda x: int(x[-16:],16) ).astype('int32')
len(aggregated_df['customer_ID'].unique())

458884

### Merging the train_data and train_labels data frames; exporting to csv

In [12]:
merged_df = pd.merge(aggregated_df, train_labels, on='customer_ID')

In [13]:
merged_df.columns

Index(['customer_ID', 'P_2_mean', 'P_2_std', 'P_2_min', 'P_2_max', 'D_39_mean',
       'D_39_std', 'D_39_min', 'D_39_max', 'B_1_mean',
       ...
       'D_143_max', 'D_144_mean', 'D_144_std', 'D_144_min', 'D_144_max',
       'D_145_mean', 'D_145_std', 'D_145_min', 'D_145_max', 'target'],
      dtype='object', length=710)

In [31]:
merged_df.to_csv("E:/COLUMBIA MS/FALL 2023/AML - Vijay Pappu/AML PROJECT/condensed_train_data.csv")

### Storing the data types of all the columns in the final data frame as they will be needed to read the new csv

In [29]:
data_types = {}
for col in merged_df.columns:
    d_type = str(type(merged_df[col][1]))
    d_type = d_type[14:-2]
    data_types[col] = d_type

print(data_types)

{'customer_ID': 'int32', 'P_2_mean': 'float64', 'P_2_std': 'float64', 'P_2_min': 'int16', 'P_2_max': 'int16', 'D_39_mean': 'float64', 'D_39_std': 'float64', 'D_39_min': 'int16', 'D_39_max': 'int16', 'B_1_mean': 'float64', 'B_1_std': 'float64', 'B_1_min': 'int16', 'B_1_max': 'int16', 'B_2_mean': 'float64', 'B_2_std': 'float64', 'B_2_min': 'int16', 'B_2_max': 'int16', 'R_1_mean': 'float64', 'R_1_std': 'float64', 'R_1_min': 'int16', 'R_1_max': 'int16', 'S_3_mean': 'float64', 'S_3_std': 'float64', 'S_3_min': 'int16', 'S_3_max': 'int16', 'D_41_mean': 'float64', 'D_41_std': 'float64', 'D_41_min': 'int16', 'D_41_max': 'int16', 'B_3_mean': 'float64', 'B_3_std': 'float64', 'B_3_min': 'int16', 'B_3_max': 'int16', 'D_42_mean': 'float64', 'D_42_std': 'float64', 'D_42_min': 'int16', 'D_42_max': 'int16', 'D_43_mean': 'float64', 'D_43_std': 'float64', 'D_43_min': 'int16', 'D_43_max': 'int16', 'D_44_mean': 'float64', 'D_44_std': 'float64', 'D_44_min': 'int16', 'D_44_max': 'int16', 'B_4_mean': 'float64

In [14]:
merged_df.sample(10)

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,D_39_mean,D_39_std,D_39_min,D_39_max,B_1_mean,...,D_143_max,D_144_mean,D_144_std,D_144_min,D_144_max,D_145_mean,D_145_std,D_145_min,D_145_max,target
348310,1109861715,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
176404,-495211633,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
322488,870435473,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,1,0.0,0.0,0,0,0.0,0.0,0,0,1
36682,-1801874012,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
73867,-1453949332,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
363686,1254173834,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
200887,-265202392,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
378620,1392992488,0.0,0.0,0,0,0.076923,0.27735,0,1,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,0
126208,-965990314,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,0,0.0,0.0,0,0,0.0,0.0,0,0,1
208330,-195484119,0.0,0.0,0,0,0.000000,0.00000,0,0,0.0,...,1,0.0,0.0,0,0,0.0,0.0,0,0,1


In [33]:
condensed_train_data = pd.read_csv("E:/COLUMBIA MS/FALL 2023/AML - Vijay Pappu/AML PROJECT/condensed_train_data.csv", dtype = data_types)

MemoryError: Unable to allocate 1.21 GiB for an array with shape (354, 458913) and data type float64

In [ ]:
condensed_train_data.sample(5)